In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
plt.rcParams.update({"font.size": 12})
%matplotlib inline

In [ ]:
!pip install -U sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv("/content/drive/MyDrive/WSB/WSB_submissions_all.csv", sep=";")
df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
df.index = pd.DatetimeIndex(df.created_utc)
df.tail()

,Unnamed: 0,author,created_utc,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio
created_utc,,,,,,,,,,,
2018-12-31 15:05:18,861747,Leading_Breath,2018-12-31 15:05:18,ab7upy,NaN,417,1,454678,NaN,"""I don't give a shit about Tesla I just want t...",0.0
2018-12-31 14:45:20,861748,skitty2,2018-12-31 14:45:20,ab7oz6,NaN,8,1,454674,The whole next year will be green.,If today closes green,0.0
2018-12-31 14:40:30,861749,Googs84,2018-12-31 14:40:30,ab7nkw,Shitpost,0,1,454672,Happy New Year to all you degenerates includin...,Happy New Year Degenerates,0.0
2018-12-31 14:34:10,861750,neocoff,2018-12-31 14:34:10,ab7lu7,NaN,7,1,454672,NaN,Chip stocks set for worst quarter in a decade....,0.0
2018-12-31 14:29:50,861751,Brodmann,2018-12-31 14:29:50,ab7km1,Discussion,2,1,454671,NaN,Aphria down 7% premarket; Questions Surface if...,0.0


In [5]:
df.drop("created_utc",axis=1)

,Unnamed: 0,author,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio
created_utc,,,,,,,,,,
2021-10-19 17:58:57,0,VendiVendiVendi,qbh10s,Gain,0,1,10967090,So 13.56 cad for BB. It’s moving and gaining ...,BB,1.0
2021-10-19 17:57:49,1,Far_Bass_7284,qbh04w,News,1,1,10967087,NaN,Surely it is a coincidence.,1.0
2021-10-19 17:55:24,2,HistoricalAnimator43,qbgyco,YOLO,0,1,10967076,NaN,yolo sava options,1.0
2021-10-19 17:54:51,3,ZaRaWi,qbgxy8,YOLO,0,1,10967076,NaN,meme,1.0
2021-10-19 17:52:33,4,CuteInterest1884,qbgw9u,DD,0,1,10967075,"BB is on the move higher, not because of the p...",BB,1.0
...,...,...,...,...,...,...,...,...,...,...
2018-12-31 15:05:18,861747,Leading_Breath,ab7upy,NaN,417,1,454678,NaN,"""I don't give a shit about Tesla I just want t...",0.0
2018-12-31 14:45:20,861748,skitty2,ab7oz6,NaN,8,1,454674,The whole next year will be green.,If today closes green,0.0
2018-12-31 14:40:30,861749,Googs84,ab7nkw,Shitpost,0,1,454672,Happy New Year to all you degenerates includin...,Happy New Year Degenerates,0.0


In [6]:
df=df.rename(columns={"Unnamed: 0": "SegmentID"})

In [7]:
df.head()

,SegmentID,author,created_utc,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio
created_utc,,,,,,,,,,,
2021-10-19 17:58:57,0,VendiVendiVendi,2021-10-19 17:58:57,qbh10s,Gain,0,1,10967090,So 13.56 cad for BB. It’s moving and gaining ...,BB,1.0
2021-10-19 17:57:49,1,Far_Bass_7284,2021-10-19 17:57:49,qbh04w,News,1,1,10967087,NaN,Surely it is a coincidence.,1.0
2021-10-19 17:55:24,2,HistoricalAnimator43,2021-10-19 17:55:24,qbgyco,YOLO,0,1,10967076,NaN,yolo sava options,1.0
2021-10-19 17:54:51,3,ZaRaWi,2021-10-19 17:54:51,qbgxy8,YOLO,0,1,10967076,NaN,meme,1.0
2021-10-19 17:52:33,4,CuteInterest1884,2021-10-19 17:52:33,qbgw9u,DD,0,1,10967075,"BB is on the move higher, not because of the p...",BB,1.0


In [29]:
#Sentences are encoded by calling model.encode() #https://www.sbert.net/
#import re
#df_title = df["title"].apply(lambda title: re.sub(r"http\S+", "", title).lower(), 1)
#df_title[5000]
#embeddings = model.encode(df_title.tolist())

In [8]:
import pickle
#Store sentences & embeddings on disc
#with open('/content/drive/MyDrive/WSB/embeddings.pkl', "wb") as fOut:
#    pickle.dump({'sentences': df_title.tolist(), 'embeddings': embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

#Load sentences & embeddings from disc
with open('/content/drive/MyDrive/WSB/embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']

In [9]:
"""
This is a more complex example on performing clustering on large scale dataset.
This examples find in a large set of sentences local communities, i.e., groups of sentences that are highly
similar. You can freely configure the threshold what is considered as similar. A high threshold will
only find extremely similar sentences, a lower threshold will find more sentence that are less similar.
A second parameter is 'min_community_size': Only communities with at least a certain number of sentences will be returned.
The method for finding the communities is extremely fast, for clustering 50k sentences it requires only 5 seconds (plus embedding comuptation).

"""
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time

titles = stored_sentences[0:40000]

print("Start clustering")
start_time = time.time()

#Two parameters to tune:
#min_cluster_size: Only consider cluster that have at least 100 elements
#threshold: Consider sentence pairs with a cosine-similarity larger than threshold as similar
clusters = util.community_detection(stored_embeddings[0:40000], min_community_size=100, threshold=0.70)

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

#Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster[0:3]:
        print("\t", titles[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", titles[sentence_id])


Start clustering
Clustering done after 33.10 sec

Cluster 1, #230 Elements 
	 to the moon
	 to the moon
	 to the moon
	 ...
	 tui me to tha moon
	 come on elon help us going to the moon
	 sdc to the moon!!!!

Cluster 2, #215 Elements 
	 adtx💎🚀🚀🚀🚀
	 💫
	 😎
	 ...
	 buy the dip🚀🚀🚀
	 wish to hell😭😭
	 stand by ✋😂

Cluster 3, #162 Elements 
	 $clov yolo
	 $clov yolo
	 $clov yolo
	 ...
	 $clov is ready again 🚀
	 copied $clov
	 $lcid $clov $idex $jagx lets goooooooo! 💰

Cluster 4, #136 Elements 
	 wsb
	 wsb now
	 new to wsb
	 ...
	 has stocktwits become the new wsb
	 he obviously found wsb.
	 can anybody guess where i found wsb?🤡

Cluster 5, #133 Elements 
	 wish$
	 $wish
	 $wish
	 ...
	 here you go you sick fucks $wish
	 started my position on $wish
	 let's try this $wish business yolo

Cluster 6, #119 Elements 
	 am i doing this right?
	 am i doing this right?
	 am i doing this right?
	 ...
	 is this how i’m supposed to do it?
	 is this right ?
	 did i do something wrong?

Cluster 7, #111 Ele

In [16]:
stored_sentences[5000]

'make the comments look like this guys search history 🦍🍌🍌🍌🍌'

In [10]:
stored_embeddings[5000]

array([-4.49486881e-01,  6.49152935e-01,  9.44378078e-02, -4.18383807e-01,
       -6.21203363e-01, -1.33402035e-01,  2.77493298e-01, -3.05665106e-01,
       -1.98207796e-03, -5.69518685e-01,  8.87069777e-02, -1.16015844e-01,
        2.45115474e-01,  1.34314358e-01,  5.40756434e-02, -2.34383941e-01,
       -2.00780764e-01, -1.81535676e-01,  5.57874776e-02, -1.54856309e-01,
       -8.97340402e-02,  1.64627269e-01,  2.08435372e-01, -1.76739633e-01,
       -3.66496563e-01, -4.10836330e-03, -2.99832970e-01,  3.33600432e-01,
        4.27890033e-01, -2.39173528e-02, -5.99784553e-02,  1.28801420e-01,
        2.72240280e-03,  2.33493552e-01,  6.88712075e-02,  3.74128580e-01,
       -3.61553639e-01,  2.59167880e-01,  3.80453497e-01,  2.19985265e-02,
        2.40873098e-01, -2.04665855e-01,  2.23402143e-01,  4.38860744e-01,
       -2.48450056e-01,  1.50471047e-01,  9.62183699e-02, -2.43318915e-01,
       -2.68962998e-02,  9.23693180e-02, -4.73714709e-01, -3.99432987e-01,
        5.43814003e-02, -

In [35]:
from sklearn.cluster import AgglomerativeClustering
# Normalize the embeddings to unit length
corpus_embeddings = stored_embeddings[0:5000] /  np.linalg.norm(stored_embeddings[0:5000], axis=1, keepdims=True)
titles = stored_sentences[0:5000]
# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None,affinity='cosine',linkage='average',  distance_threshold=0.90) #,affinity='cosine', linkage='average', distance_threshold=1
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(titles[sentence_id])
print(len(clustered_sentences.items()))
for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(len(cluster))
    print(cluster)
    print("")

24
Cluster  2
2945
['bb', 'yolo sava options', 'bb', 'sdig', 'prog yolo 1.5 yrs out', "don't be a menace to hedge funds by squeezing their shorts of $hood", '$372k gains from tsla deliveries and pre-earnings', 'tpx, look for this one in the future and help a bit lol 😂', '💎🤲🏾 bb 🚀🚀🚀🚀🚀', "serious question: trying to understand why amc and gme are still pushed here and others that have yet to squeeze aren't? wasn't the goal fulfilled?", 'tsla earnings prediction: they beat estimates, but will this be enough?', 'bowx we spac merger dd - wework', 'hedge funds are piling into playtika holding corp. (pltk)', '$adxs continue to buy more and gain more', 'tilray “taking off” out of nowhere? what’s y’all thoughts.', 'btzi. is anybody watching this?', '$tlry is moving big today. all is weed smokers rejoice.', 'how high are we getting today? no pun intended.', 'looking for advice', 'blackberry granted new us patent titled "method and system for trailer tracking and inventory management" - $bb up 6%